# Semantic Segmentation 

This project detects road and not-road using a dataset and tensorflow. 

## Imports and gpu-check

In [1]:
#!/usr/bin/env python3
import time
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests
from datetime import timedelta
import scipy.misc

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))


TensorFlow Version: 1.10.0
Default GPU Device: /device:GPU:0


## Parameters

In [2]:
L2_REG = 1e-5 
STDDEV = 1e-2
KEEP_PROB = 0.5
LEARNING_RATE = 1e-3
EPOCHS = 2
BATCH_SIZE = 4 
IMAGE_SHAPE = (160, 576)
NUM_CLASSES = 2

DATA_DIR = './data'
RUNS_DIR = './runs'
MODEL_DIR = './models'

## load_vgg

In [3]:
def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
    
    graph = tf.get_default_graph()

    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    image_input = graph.get_tensor_by_name(vgg_input_tensor_name)
    keep_prob = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    layer3 = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    layer4 = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    layer7 = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)
    return image_input, keep_prob, layer3, layer4, layer7

print("Load vgg16 model: ")
tests.test_load_vgg(load_vgg, tf)


Load vgg16 model: 
Tests Passed


## layers()

In [4]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes): 
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer3_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer7_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    kernel_regularizer = tf.contrib.layers.l2_regularizer(L2_REG)
    kernel_initializer = tf.random_normal_initializer(stddev=STDDEV)
    
    # perform a convolution on layer 7 and connect it to the output
    layer7_conv_1x1 = tf.layers.conv2d(vgg_layer7_out, num_classes, 1,1, padding = 'same',
                                kernel_initializer= kernel_initializer, 
                                kernel_regularizer= kernel_regularizer)
    out = tf.layers.conv2d_transpose(layer7_conv_1x1, num_classes, 4, 2, padding ='same', 
                                kernel_initializer= kernel_initializer, 
                                kernel_regularizer= kernel_regularizer)
    
    # perform a convolution on layer 4 and add it to the existing output, then do a transposed conv
    layer4_conv_1x1 = tf.layers.conv2d(vgg_layer4_out, num_classes, 1,1, padding = 'same',
                                kernel_initializer= kernel_initializer, 
                                kernel_regularizer= kernel_regularizer)
    out = tf.add(out, layer4_conv_1x1)
    out = tf.layers.conv2d_transpose(out, num_classes, 4, 2, padding='same', 
                                kernel_initializer= kernel_initializer, 
                                kernel_regularizer= kernel_regularizer)
                                                                             
    # perform a convolution on layer 3 and add it to the existing output, then do a transposed conv
    layer3_conv_1x1 = tf.layers.conv2d(vgg_layer3_out, num_classes, 1,1, padding = 'same',
                                kernel_initializer= kernel_initializer, 
                                kernel_regularizer= kernel_regularizer)
    out = tf.add(out, layer3_conv_1x1)
    out = tf.layers.conv2d_transpose(out, num_classes, 16, 8, padding='same',                                       
                                kernel_initializer= kernel_initializer, 
                                kernel_regularizer= kernel_regularizer)
    
    return out
print("Load layers: ")
tests.test_layers(layers)

Load layers: 
Tests Passed


## optimize

In [5]:

def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    logits = tf.reshape(nn_last_layer, (-1, num_classes))
    labels = tf.reshape(correct_label, (-1, num_classes))
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=correct_label))
    train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy_loss)
    
    return logits, train_op, cross_entropy_loss


tests.test_optimize(optimize)


Tests Passed


## train_nn

In [8]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate, saver, data_dir):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    for epoch in range(epochs):
        s_time = time.time()
        for image, gt_image in get_batches_fn(batch_size):
            _, loss = sess.run([train_op, cross_entropy_loss ], 
                                feed_dict = {input_image: image, 
                                             correct_label: gt_image, 
                                             keep_prob: KEEP_PROB,
                                             learning_rate: LEARNING_RATE})
            iou, _ = sess.run([mIoU, confusionMatrix])
        # Print data on the learning process
        print("Epoch: {}".format(epoch + 1), "/ {}".format(epochs), 
              " Loss: {:.3f}".format(loss),
              " IoU : {:.3f}".format(iou),
              " Time: ", str(timedelta(seconds=(time.time() - s_time))))
        
        # save the model at the end
        if (epoch + 1) == epochs: 
            save_path = saver.save(sess, os.path.join(data_dir, 'epoch_' + str(epoch) + '.ckpt'))
            helper.save_inference_samples(RUNS_DIR, DATA_DIR, sess, IMAGE_SHAPE, logits, keep_prob, input)


## run

In [ ]:
def run():
    tests.test_for_kitti_dataset(DATA_DIR)
    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(DATA_DIR)

    print("Training...")

    with tf.Session() as sess:
        # get vgg model
        vgg_path = os.path.join(DATA_DIR, 'vgg')
        # use helper function
        get_batches_fn = helper.gen_batch_function(os.path.join(DATA_DIR, 'data_road/training'), IMAGE_SHAPE)
        # Added augmentation using random flip and brightness/contrast changes, see helper.py
        
        # Loadd vgg, and pin in layers
        input, keep_prob, layer3, layer4, layer7 = load_vgg(sess, vgg_path)
        # Load encoder and connecto to output
        out = layers(layer3, layer4, layer7, NUM_CLASSES)
        
        # Placeholder for variable resolution images
        correct_label = tf.placeholder(dtype = tf.float32, shape = (None, None, None, NUM_CLASSES))
        learning_rate = tf.placeholder(dtype = tf.float32)
        
        # call function to add an optimizer to the net 
        logits, train_op, cross_entropy_loss = optimize(out, correct_label, learning_rate, NUM_CLASSES)
        
        # set seed across all sessions so results are comparable
        tf.set_random_seed(123)
        
        # set the iou metric 
        mIoU, confusionMatrix = tf.contrib.metrics.streaming_mean_iou(out,correct_label,NUM_CLASSES)
        
        sess.run(tf.global_variables_initializer())
        
        #saver object to save states
        saver = tf.train.Saver() 
        
        #train the net
        train_nn(sess, EPOCHS, BATCH_SIZE, get_batches_fn, train_op, cross_entropy_loss, input, correct_label,
                 keep_prob, learning_rate,  saver, MODEL_DIR)
        
        # Apply the net to images
        helper.save_inference_samples(RUNS_DIR, DATA_DIR, sess, IMAGE_SHAPE, logits, keep_prob, input)
        print("... done")
        
run()

Tests Passed
Training...
INFO:tensorflow:Restoring parameters from ./data\vgg\variables\variables


# log

### without regularization

Epoch: 1 / 5  Loss: 0.947  Time:  0:01:31.879395
Epoch: 2 / 5  Loss: 0.774  Time:  0:01:29.919057
Epoch: 3 / 5  Loss: 0.726  Time:  0:01:40.409835
Epoch: 4 / 5  Loss: 0.700  Time:  0:01:56.143895
Epoch: 5 / 5  Loss: 0.672  Time:  0:01:58.565175
Training Finished. Saving test images to: ./runs\1540840717.903083


### standard regularization

Epoch: 1 / 20  Loss: 0.135  Time:  0:01:42.530179
Epoch: 2 / 20  Loss: 0.255  Time:  0:02:06.080995
Epoch: 3 / 20  Loss: 0.089  Time:  0:02:09.374919
Epoch: 4 / 20  Loss: 0.073  Time:  0:02:09.156908
Epoch: 5 / 20  Loss: 0.075  Time:  0:02:09.304775
Epoch: 6 / 20  Loss: 0.083  Time:  0:02:10.274703
Epoch: 7 / 20  Loss: 0.135  Time:  0:02:09.564648
Epoch: 8 / 20  Loss: 0.036  Time:  0:02:09.255591
Epoch: 9 / 20  Loss: 0.044  Time:  0:02:09.056574
Epoch: 10 / 20  Loss: 0.090  Time:  0:02:09.841141
Epoch: 11 / 20  Loss: 0.049  Time:  0:02:08.956540
Epoch: 12 / 20  Loss: 0.048  Time:  0:02:09.494981
Epoch: 13 / 20  Loss: 0.067  Time:  0:02:09.884587
Epoch: 14 / 20  Loss: 0.015  Time:  0:02:09.849671
Epoch: 15 / 20  Loss: 0.026  Time:  0:02:10.587436
Epoch: 16 / 20  Loss: 0.089  Time:  0:02:09.789505
Epoch: 17 / 20  Loss: 0.038  Time:  0:02:09.382614
Epoch: 18 / 20  Loss: 0.049  Time:  0:02:09.362768
Epoch: 19 / 20  Loss: 0.063  Time:  0:02:09.163698
Epoch: 20 / 20  Loss: 0.052  Time:  0:02:09.201310
1540845167.5800643

### augment with flip and random brightness and contrast 


Epoch: 1 / 20  Loss: 0.746  Time:  0:01:31.271338
Epoch: 2 / 20  Loss: 0.586  Time:  0:01:32.809698
Epoch: 3 / 20  Loss: 0.457  Time:  0:01:52.036394
Epoch: 4 / 20  Loss: 0.320  Time:  0:02:04.046675
Epoch: 5 / 20  Loss: 0.165  Time:  0:02:09.064407
Epoch: 6 / 20  Loss: 0.129  Time:  0:02:11.825765
Epoch: 7 / 20  Loss: 0.080  Time:  0:02:08.195775
Epoch: 8 / 20  Loss: 0.102  Time:  0:02:08.505612
Epoch: 9 / 20  Loss: 0.117  Time:  0:02:08.363144
Epoch: 10 / 20  Loss: 0.073  Time:  0:02:08.413695
Epoch: 11 / 20  Loss: 0.059  Time:  0:02:04.330584
Epoch: 12 / 20  Loss: 0.099  Time:  0:02:08.333077
Epoch: 13 / 20  Loss: 0.078  Time:  0:02:09.456341
Epoch: 14 / 20  Loss: 0.086  Time:  0:02:09.155645
Epoch: 15 / 20  Loss: 0.072  Time:  0:02:18.026403
Epoch: 16 / 20  Loss: 0.053  Time:  0:02:09.169739
Epoch: 17 / 20  Loss: 0.091  Time:  0:02:18.399122
Epoch: 18 / 20  Loss: 0.078  Time:  0:02:14.184786
Epoch: 19 / 20  Loss: 0.136  Time:  0:02:17.844234
Epoch: 20 / 20  Loss: 0.191  Time:  0:02:13.351590
Training Finished. Saving test images to: ./runs\1540852572.6791086


looks like too little generalization or too little regularization


### regularization e-3 --> e-5 gets slightly better in iou
Epoch: 1 / 20  Loss: 0.790  Time:  0:01:33.200101
Epoch: 2 / 20  Loss: 0.691  Time:  0:01:23.696226
Epoch: 3 / 20  Loss: 0.571  Time:  0:01:32.057452
Epoch: 4 / 20  Loss: 0.394  Time:  0:01:38.021367
Epoch: 5 / 20  Loss: 0.449  Time:  0:01:46.768626
Epoch: 6 / 20  Loss: 0.259  Time:  0:01:54.783575
Epoch: 7 / 20  Loss: 0.113  Time:  0:01:55.703466
Epoch: 8 / 20  Loss: 0.089  Time:  0:01:59.253920
Epoch: 9 / 20  Loss: 0.162  Time:  0:02:04.317690
Epoch: 10 / 20  Loss: 0.128  Time:  0:02:07.141466
Epoch: 11 / 20  Loss: 0.110  Time:  0:01:54.742900
Epoch: 12 / 20  Loss: 0.222  Time:  0:02:16.932343
Epoch: 13 / 20  Loss: 0.105  Time:  0:02:09.088753
Epoch: 14 / 20  Loss: 0.156  Time:  0:02:08.976205
Epoch: 15 / 20  Loss: 0.043  Time:  0:02:09.306973
Epoch: 16 / 20  Loss: 0.132  Time:  0:02:09.489564
Epoch: 17 / 20  Loss: 0.050  Time:  0:02:09.450222
Epoch: 18 / 20  Loss: 0.100  Time:  0:02:09.112877
Epoch: 19 / 20  Loss: 0.124  Time:  0:02:09.056755
Epoch: 20 / 20  Loss: 0.159  Time:  0:02:09.210153
Training Finished. Saving test images to: ./runs\1540886071.6455922
Done

### learning rate 1e-5 --> 1e-3 
Epoch: 1 / 20  Loss: 0.217  Time:  0:01:31.703709
Epoch: 2 / 20  Loss: 0.120  Time:  0:01:22.339839
Epoch: 3 / 20  Loss: 0.191  Time:  0:01:22.125157
Epoch: 4 / 20  Loss: 0.080  Time:  0:01:22.190095
Epoch: 5 / 20  Loss: 0.101  Time:  0:01:22.078166
Epoch: 6 / 20  Loss: 0.111  Time:  0:01:22.082627
Epoch: 7 / 20  Loss: 0.150  Time:  0:01:22.262901
Epoch: 8 / 20  Loss: 0.049  Time:  0:01:23.648054
Epoch: 9 / 20  Loss: 0.068  Time:  0:01:24.395504
Epoch: 10 / 20  Loss: 0.053  Time:  0:01:25.935247
Epoch: 11 / 20  Loss: 0.066  Time:  0:01:23.495294
Epoch: 12 / 20  Loss: 0.066  Time:  0:01:26.084785
Epoch: 13 / 20  Loss: 0.061  Time:  0:01:28.843525
Epoch: 14 / 20  Loss: 0.033  Time:  0:01:29.724895
Epoch: 15 / 20  Loss: 0.031  Time:  0:01:33.120557
Epoch: 16 / 20  Loss: 0.076  Time:  0:01:34.574232
Epoch: 17 / 20  Loss: 0.041  Time:  0:01:35.779546
Epoch: 18 / 20  Loss: 0.049  Time:  0:01:36.263968
Epoch: 19 / 20  Loss: 0.037  Time:  0:01:36.678812
Epoch: 20 / 20  Loss: 0.027  Time:  0:01:37.031775
Training ./runs\1540902476.2898514

###   -25, 25 --> -50, 50 brightness augmentation because shadows are a major struggle. May also be caused by bad image quality or too little training data and reg e-4 (too much? 
good loss but problems with shadows

Epoch: 1 / 20  Loss: 0.422  Time:  0:01:32.222510
Epoch: 2 / 20  Loss: 0.157  Time:  0:01:34.046224
Epoch: 3 / 20  Loss: 0.115  Time:  0:01:53.419229
Epoch: 4 / 20  Loss: 0.106  Time:  0:02:07.891763
Epoch: 5 / 20  Loss: 0.052  Time:  0:02:09.694977
Epoch: 6 / 20  Loss: 0.097  Time:  0:02:09.440225
Epoch: 7 / 20  Loss: 0.095  Time:  0:02:17.469924
Epoch: 8 / 20  Loss: 0.114  Time:  0:02:09.271687
Epoch: 9 / 20  Loss: 0.058  Time:  0:02:18.333528
Epoch: 10 / 20  Loss: 0.110  Time:  0:02:08.726469
Epoch: 11 / 20  Loss: 0.097  Time:  0:02:10.661361
Epoch: 12 / 20  Loss: 0.038  Time:  0:02:20.865498
Epoch: 13 / 20  Loss: 0.027  Time:  0:02:21.144119
Epoch: 14 / 20  Loss: 0.093  Time:  0:02:12.491166
Epoch: 15 / 20  Loss: 0.084  Time:  0:02:15.883092
Epoch: 16 / 20  Loss: 0.033  Time:  0:02:20.518270
Epoch: 17 / 20  Loss: 0.039  Time:  0:02:21.881451
Epoch: 18 / 20  Loss: 0.036  Time:  0:02:16.813000
Epoch: 19 / 20  Loss: 0.050  Time:  0:02:13.290732
Epoch: 20 / 20  Loss: 0.050  Time:  0:02:17.622825
Training Finished. Saving test images to: ./runs\1540914346.7242293


### agressive brightness augmentation with reg e-3 


Epoch: 1 / 20  Loss: 0.329  Time:  0:01:36.565431
Epoch: 2 / 20  Loss: 0.145  Time:  0:01:42.483178
Epoch: 3 / 20  Loss: 0.178  Time:  0:02:03.556036
Epoch: 4 / 20  Loss: 0.145  Time:  0:02:11.861257
Epoch: 5 / 20  Loss: 0.048  Time:  0:02:13.503376
Epoch: 6 / 20  Loss: 0.051  Time:  0:02:13.867488
Epoch: 7 / 20  Loss: 0.038  Time:  0:02:19.680010
Epoch: 8 / 20  Loss: 0.023  Time:  0:02:19.636314
Epoch: 9 / 20  Loss: 0.073  Time:  0:02:11.722002
Epoch: 10 / 20  Loss: 0.056  Time:  0:02:19.195323
Epoch: 11 / 20  Loss: 0.239  Time:  0:02:12.308154
Epoch: 12 / 20  Loss: 0.070  Time:  0:02:21.041252
Epoch: 13 / 20  Loss: 0.198  Time:  0:02:20.435198
Epoch: 14 / 20  Loss: 0.045  Time:  0:02:16.609337
Epoch: 15 / 20  Loss: 0.071  Time:  0:02:14.922345
Epoch: 16 / 20  Loss: 0.114  Time:  0:02:23.148407
Epoch: 17 / 20  Loss: 0.070  Time:  0:02:22.312371
Epoch: 18 / 20  Loss: 0.078  Time:  0:02:22.637378
Epoch: 19 / 20  Loss: 0.079  Time:  0:02:23.331576
Epoch: 20 / 20  Loss: 0.133  Time:  0:02:30.637584

### reduced dropout of 0.8 produces a lot false positive areas 


### final parameters: 
L2_REG = 1e-5 
STDEV = 1e-2
KEEP_PROB = 0.5
LEARNING_RATE = 1e-3
EPOCHS = 20
BATCH_SIZE = 4 
IMAGE_SHAPE = (160, 576)
NUM_CLASSES = 2


Epoch: 1 / 20  Loss: 0.289  Time:  0:01:38.017160
Epoch: 2 / 20  Loss: 0.223  Time:  0:02:06.169096
Epoch: 3 / 20  Loss: 0.150  Time:  0:02:14.851168
Epoch: 4 / 20  Loss: 0.206  Time:  0:02:23.226233
Epoch: 5 / 20  Loss: 0.787  Time:  0:02:13.511161
Epoch: 6 / 20  Loss: 0.130  Time:  0:02:16.925585
Epoch: 7 / 20  Loss: 0.086  Time:  0:02:20.199117
Epoch: 8 / 20  Loss: 0.059  Time:  0:02:19.996068
Epoch: 9 / 20  Loss: 0.143  Time:  0:02:21.412368
Epoch: 10 / 20  Loss: 0.203  Time:  0:02:20.029864
Epoch: 11 / 20  Loss: 0.118  Time:  0:02:12.562245
Epoch: 12 / 20  Loss: 0.077  Time:  0:02:22.055017
Epoch: 13 / 20  Loss: 0.204  Time:  0:02:15.680001
Epoch: 14 / 20  Loss: 0.084  Time:  0:02:18.484648
Epoch: 15 / 20  Loss: 0.135  Time:  0:02:22.618080
Epoch: 16 / 20  Loss: 0.025  Time:  0:02:20.716430
Epoch: 17 / 20  Loss: 0.043  Time:  0:02:11.576349
Epoch: 18 / 20  Loss: 0.114  Time:  0:02:21.392957
Epoch: 19 / 20  Loss: 0.059  Time:  0:02:22.356014
Epoch: 20 / 20  Loss: 0.089  Time:  0:02:22.682873
Training Finished. Saving test images to: ./runs\1540929179.4676092
Done